In [18]:
import mwu_measures
import pandas as pd
import numpy as np
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher

mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
this_corpus = mwu_measures.processing_corpus.process_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=True)
ngram_selection = mwu_examples['Expression'].tolist()
ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)

90357 lines processed
178289 lines processed
287795 lines processed
301046 lines processed
we bring the boom


In [19]:
helper = Fetcher(this_corpus.corpus_conn)

In [20]:
x = helper.fetch_bigrams(ngram_selection, 'bw')

AttributeError: 'list' object has no attribute 'split'

In [25]:
trigrams = [[' '.join(ngram), ngram[0], ngram[1]] for ngram in ngram_selection if len(ngram) == 3]

In [26]:
trigrams = pd.DataFrame(trigrams, columns = ['query', 'ug_1', 'ug_2'])

In [28]:
this_corpus.corpus_conn.execute(
    """
    CREATE OR REPLACE TEMPORARY TABLE this_query (query TEXT, ug_1 TEXT, ug_2 TEXT, hash_1 UINT64, hash_2 UINT64)
""")

In [29]:
this_corpus.corpus_conn.execute(f"""
    INSERT INTO this_query SELECT *, hash(ug_1) as hash_1, hash(ug_2) as hash_2 FROM trigrams
""")

In [31]:
this_corpus.corpus_conn.execute("SELECT * FROM this_query").fetch_df()

,query,ug_1,ug_2,hash_1,hash_2
0,‘tis the season,‘tis,the,18217765797884696497,1313387571633107563
1,8 ball pool,8,ball,16907281043801152130,260354296636108172
2,a bad break,a,bad,584566915021039271,11530801326569434646
3,a big ask,a,big,584566915021039271,16477750156076519317
4,a big deal,a,big,584566915021039271,16477750156076519317
...,...,...,...,...,...
9722,zest for life,zest,for,17948273043288103573,4899634184053917532
9723,zip one's lip,zip,one's,10025808472108214766,9092675030052912959
9724,zip your lip,zip,your,10025808472108214766,17821877056886481474
9725,zone in on,zone,in,10517493854241104905,14322568082006533075


In [35]:
this_corpus.corpus_conn.execute(
    f"""
    CREATE OR REPLACE TABLE filtered_db AS
    SELECT
        corpus,
        hash(ug_1) as hash_1,
        hash(ug_2) as hash_2,
        ug_3
    FROM trigram_db
    WHERE 
        hash_1 IN (
            SELECT hash_1 FROM this_query
        ) AND
        hash_2 IN (
            SELECT hash_2 from this_query
        )
""")

In [43]:
this_corpus.corpus_conn.execute(
    f"""
    CREATE OR REPLACE TABLE joined_db AS                  
    SELECT 
        query,
        corpus,
        this_query.ug_1,
        this_query.ug_2,
        filtered_db.ug_3,
        SUM(freq) as freq
    FROM this_query
    INNER JOIN filtered_db
    ON filtered_db.hash_1 = this_query.hash_1 AND filtered_db.hash_2 = this_query.hash_2
    GROUP BY
        query,
        corpus,
        ug_1,
        ug_2,
        ug_3
""")

In [46]:
this_corpus.corpus_conn.execute("SELECT * FROM joined_db").fetch_df().sort_values(by=['freq'], ascending = False)

,query,corpus,ug_1,ug_2,ug_3,freq
1615864,i don't know,K,i,don't,know,252.0
1923844,i don't care,K,i,don't,know,252.0
1512258,i don't understand,K,i,don't,know,252.0
3058936,a lot of,K,a,lot,of,199.0
612748,a number of,H,a,number,of,199.0
...,...,...,...,...,...,...
1236906,in a stitherum,G,in,a,situation,1.0
1236907,in a stitherum,G,in,a,student's,1.0
1236908,in a stitherum,G,in,a,treaty,1.0
1236909,in a stitherum,G,in,a,union,1.0


In [11]:
x['space suit']

{'G': {'his': 1.0,
  'lightweight': 1.0,
  "won't": 1.0,
  'to': 8.0,
  'follow': 2.0,
  'taekwondo': 1.0,
  'a': 1.0,
  'purple': 1.0,
  'safari': 1.0,
  'evening': 1.0,
  '‘': 1.0,
  'flannel': 1.0,
  'threepiece': 1.0,
  'smartest': 1.0,
  'acrylic': 1.0,
  'followed': 1.0},
 'F': {'sabbath': 1.0,
  'his': 1.0,
  'to': 6.0,
  'the': 3.0,
  'followed': 2.0,
  'pinstriped': 2.0,
  'silver': 1.0,
  'wool': 1.0,
  'best': 1.0,
  'a': 1.0,
  'bringing': 1.0,
  'navyblue': 1.0,
  'would': 1.0,
  'festival': 1.0},
 'E': {'to': 6.0,
  'only': 1.0,
  'work': 1.0,
  'a': 1.0,
  'that': 1.0,
  'wool': 1.0},
 'J': {'her': 1.0,
  'should': 1.0,
  'the': 2.0,
  'a': 2.0,
  "wouldn't": 1.0,
  'to': 1.0,
  'floral': 1.0,
  'his': 1.0,
  'american': 1.0,
  'leather': 1.0,
  'linen': 1.0,
  'row': 1.0},
 'B': {'will': 1.0,
  'pants': 1.0,
  'to': 4.0,
  'follow': 2.0,
  'track': 1.0,
  'twopiece': 1.0,
  'a': 1.0,
  'law': 1.0,
  'may': 1.0,
  'which': 1.0,
  'interviewing': 1.0,
  'silk': 1.0,
  'do